## Problem Description  
Determine the optimal server facilities to upgrade while upgrading at least 90 servers of each server type while minimizing total upgrade cost.

Data Center Variables  

|Data Center |Cost ($M) |Secure Servers |Super Secure Servers |
|------------|----------|---------------|---------------------|
|Michigan    |$2.5 	    |50	            |30                   |
|New York 1	 |$3.5 	    |80	            |40                   |
|New York 2	 |$3.5      |40	            |80                   |
|Ohio	     |$4.0 	    |90	            |60                   |
|California	 |$2.0 	    |20	            |30                   |




In [1]:
import numpy as np 
import pandas as pd
import pulp


In [7]:
#fixed variables
data_center = ['Michigan', 'New York 1', 'New York 2', 'Ohio', 'California']
cost = np.array([2.5, 3.5, 3.5, 4.0, 2.0])
secure_servers = np.array([50, 80, 40, 90, 20])
super_secure_servers = np.array([30, 40, 80, 60, 30])

min_servers = 90

In [27]:
#create the model
model = pulp.LpProblem(name='uprade_schedule', sense=pulp.LpMinimize)


dc_variables = pulp.LpVariable.matrix('DC', data_center,
                                             lowBound=0, cat=pulp.LpBinary)

#Sum of secure servers must be at least 90
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(secure_servers*dc_variables) ,
    sense=pulp.LpConstraintGE,
    name='min_secure_servers',
    rhs=min_servers))

#Sum of super secure servers must be at least 90
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(super_secure_servers*dc_variables) ,
    sense=pulp.LpConstraintGE,
    name='min_super_secure_servers',
    rhs=min_servers))

objective = pulp.lpSum(cost*dc_variables)

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    print('Upgrade: ', end='')
    for i,var in enumerate(dc_variables):
        if var.value() == 1.0:
            print(f' {data_center[i]}', end='')
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $6
Upgrade:  Michigan New York 2